In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy import optimize

In [2]:
# define function for performing kubic harmonic regression
def kubic_harm(prop,norm): #order is in (111), (100), and (110)
    x1 = [0.333333333,1,0.5]
    x2 = [0.037037037,0,0]
    fit_x = np.array([x1,x2]).transpose()
    model = LinearRegression()
    model.fit(fit_x,[prop[ii]/norm[ii] for ii in range(0,len(prop))])
    return [model.intercept_,model.coef_[0],model.coef_[1]]

In [3]:
def NP_plot(ph1,ph2,Z,sys_name,p,surf_name,item,scale_min,scale_max,dpi_set,colmap):
    fig, ax = plt.subplots(subplot_kw=dict(projection='polar'),figsize=(9,8))
    im = ax.contourf(ph1,ph2,Z,vmin=scale_min,vmax=scale_max,cmap=colmap)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    plt.setp(ax.spines.values(), linewidth=1.5)
    fig.colorbar(im)
    ax.set_xticks([])
    ax.set_yticks([])
    fig.set(facecolor='white')
    fig.savefig(p+surf_name+sys_name+str(item)+".png",dpi=dpi_set)
    plt.close(fig)

In [4]:
# define function for vibration partition function calculator
def qvib_calc(vibs,T):
    kB = 8.617333262145*10**(-5) # eV/K, Boltzmann Constant
    qvib = 1
    for item in vibs:
        if item != 0:
            temp = np.exp(-item/2/kB/T/1000)/(1 - np.exp(-item/kB/T/1000))
            qvib = qvib*temp
    return qvib

In [5]:
def NP_calc(npoints,T,yO2,yH2,P0,O_kB,O_kC,O_kmA,O_kmB,O_kmC,O_kbA,O_kbB,O_kbC,O_kB2,O_kC2,
            OH_kC,OH_kbA,OH_kbB,OH_kbC,OH_kC2):

    phi = np.radians(np.linspace(0,360,npoints))
    theta = np.radians(np.linspace(0,90,npoints))

    P0_H2 = yH2*P0 # Pa, initial H2 partial pressures
    P0_O2 = yO2*P0 # Pa, initial O2 partial pressures
    P_inert = (1 - yH2 - yO2)*P0 # Pa, accounts for any inert gas species present
    ext_rxn = min(2*P0_O2,P0_H2)*0.99 # Pa, extent of reaction assumed to reaction near 99% of equilibrium 
    P_H2 = (P0_H2 - ext_rxn)/(P0_H2 + P0_O2 + P_inert - 1/2*ext_rxn)*P0 # Pa, near equil. H2 partial pressure
    P_O2 = (P0_O2 - 1/2*ext_rxn)/(P0_H2 + P0_O2 + P_inert - 1/2*ext_rxn)*P0 # Pa, near equil. O2 partial pressure
    P_H2O = (ext_rxn)/(P0_H2 + P0_O2 + P_inert - 1/2*ext_rxn)*P0 # Pa, near equil. H2O partial pressure
    
    kB = 1.38064852*10**(-23) # J/K, Boltzmann Constant
    h = 6.62607004*10**(-34) # J*s, Plank's Constant
    mH2 = 1.00784*2/(6.022*10**23)/1000 # kg, H2 Mass
    mO2 = 31.999/(6.022*10**23)/1000 # kg, O2 Mass
    mH2O = 18.01528/(6.022*10**23)/1000 # kg, H2O Mass
    kB2 = 8.617333262145*10**(-5) # eV/K, Boltzmann Constant
    vH2 = [537.767435] # meV, H2 bond frequency in gas phase
    vO2 = [191.125756] # meV, O2 bond frequency in gas phase
    vH2O = [473.441566,459.393964,198.081414] # meV, H2O bond freqency in gas phase
    I_H2 = mH2/4*(0.747277565*10**(-10))**2 # kg*m^2, H2 moment of inertia
    I_O2 = mO2/4*(1.239620627*10**(-10))**2 # kg*m^2, O2 moment of inertia
    I_H2O = 6.3848765503633E-141 # kg^3*m^6, H2O moment of inertia
    lamb_H = h/(2*np.pi*mH2*kB*T)**(1/2)
    lamb_O = h/(2*np.pi*mO2*kB*T)**(1/2)
    lamb_H2O = h/(2*np.pi*mH2O*kB*T)**(1/2)
    
    Zrot_H2 = (8*np.pi**2*I_H2*kB*T/2/h**2) # H2 rotational partition function
    Zrot_O2 = (8*np.pi**2*I_O2*kB*T/2/h**2) # O2 rotational partition function
    Zrot_H2O = 8*np.pi**2/(2*h**3)*(2*np.pi*kB*T)**(3/2)*I_H2O**(1/2) # H2O rotational partition function
    Zvib_H2 = qvib_calc(vH2,T) # H2 vibrational partition function
    Zvib_O2 = qvib_calc(vO2,T) # O2 vibrational partition function
    Zvib_H2O = qvib_calc(vH2O,T) # O2 vibrational partition function
    Ztrans_H2 = kB*T/P_H2*1/lamb_H**3 # H2 translational partition function
    Ztrans_O2 = kB*T/P_O2*1/lamb_O**3 # O2 translational partition function
    Ztrans_H2O = kB*T/P_H2O*1/lamb_H2O**3 # H2O translational partition function
    
    covO = []
    covOH = []
    Aform_O = []
    Aform_OH = []
    Aform_S = []
    for ii in range(0,npoints): # loop over all theta values
        for jj in range(0,npoints): # loop over all phi values        
            nx = np.sin(theta[ii])*np.cos(phi[jj])
            ny = np.sin(theta[ii])*np.sin(phi[jj])
            nz = np.cos(theta[ii])
            
            # calculate Eads vs coverage slope and intercept at point on NP
            Eads_O_B = O_kB[0] + O_kB[1]*(nx**4+ny**4+nz**4) + O_kB[2]*(nx**2*ny**2*nz**2)
            Eads_O_C = O_kC[0] + O_kC[1]*(nx**4+ny**4+nz**4) + O_kC[2]*(nx**2*ny**2*nz**2)
            
            Eads_OH_C = OH_kC[0] + OH_kC[1]*(nx**4+ny**4+nz**4) + OH_kC[2]*(nx**2*ny**2*nz**2)
            
            # calculate the best fit slope and intercept for Gvib(O/S) with coverage at point on NP
            O_Avib_mA = O_kmA[0] + O_kmA[1]*(nx**4+ny**4+nz**4) + O_kmA[2]*(nx**2*ny**2*nz**2)
            O_Avib_mB = O_kmB[0] + O_kmB[1]*(nx**4+ny**4+nz**4) + O_kmB[2]*(nx**2*ny**2*nz**2)
            O_Avib_mC = O_kmC[0] + O_kmC[1]*(nx**4+ny**4+nz**4) + O_kmC[2]*(nx**2*ny**2*nz**2)
            
            O_Avib_bA = O_kbA[0] + O_kbA[1]*(nx**4+ny**4+nz**4) + O_kbA[2]*(nx**2*ny**2*nz**2)
            O_Avib_bB = O_kbB[0] + O_kbB[1]*(nx**4+ny**4+nz**4) + O_kbB[2]*(nx**2*ny**2*nz**2)
            O_Avib_bC = O_kbC[0] + O_kbC[1]*(nx**4+ny**4+nz**4) + O_kbC[2]*(nx**2*ny**2*nz**2)
            
            OH_Avib_bA = OH_kbA[0] + OH_kbA[1]*(nx**4+ny**4+nz**4) + OH_kbA[2]*(nx**2*ny**2*nz**2)
            OH_Avib_bB = OH_kbB[0] + OH_kbB[1]*(nx**4+ny**4+nz**4) + OH_kbB[2]*(nx**2*ny**2*nz**2)
            OH_Avib_bC = OH_kbC[0] + OH_kbC[1]*(nx**4+ny**4+nz**4) + OH_kbC[2]*(nx**2*ny**2*nz**2)
            
            # find equilibrium coverage at point on NP
            Nsites = 4
            Eads_OH = Eads_OH_C
            OH_Avib_T = (OH_Avib_bA)*T**2 + (OH_Avib_bB)*T + (OH_Avib_bC)
            Gads_OH = (Eads_OH + OH_Avib_T - 1/2*(-kB2*T*np.log(Ztrans_O2*Zvib_O2*Zrot_O2*Ztrans_H2*Zvib_H2*Zrot_H2)))/Nsites
            Ke_OH = np.exp(-Gads_OH/kB2/T)
            A = Ke_OH*P_O2**(1/2)*P_H2**(1/2)
            def func(xO):
                Eads_O = Eads_O_B*xO + Eads_O_C
                O_Avib_T = (O_Avib_mA*xO+O_Avib_bA)*T**2 + (O_Avib_mB*xO+O_Avib_bB)*T + (O_Avib_mC*xO+O_Avib_bC)
                Gads_O = (Nsites*xO*Eads_O + O_Avib_T - Nsites*xO/2*(-kB2*T*np.log(Ztrans_O2*Zvib_O2*Zrot_O2)))/Nsites
                Ke_O = np.exp(-Gads_O/kB2/T)
                B = Ke_O*P_O2**(1/2)
                fO = B*(1 - A/(A+1)) + xO*(B*A/(A+1) - B - 1)
                return fO
            xn_O = optimize.brentq(func,0,1.001)
            xn_OH = A*(1-xn_O)/(A+1)
            
            ## Aform values
            Ef_O_B = O_kB2[0] + O_kB2[1]*(nx**4+ny**4+nz**4) + O_kB2[2]*(nx**2*ny**2*nz**2)
            Ef_O_C = O_kC2[0] + O_kC2[1]*(nx**4+ny**4+nz**4) + O_kC2[2]*(nx**2*ny**2*nz**2)
            
            Ef_OH_C = OH_kC2[0] + OH_kC2[1]*(nx**4+ny**4+nz**4) + OH_kC2[2]*(nx**2*ny**2*nz**2)
            
            Eform_O = Ef_O_B*xn_O + Ef_O_C
            Eform_OH = Ef_OH_C

            O_Avib_T = (O_Avib_mA*xn_O+O_Avib_bA)*T**2 + (O_Avib_mB*xn_O+O_Avib_bB)*T + (O_Avib_mC*xn_O+O_Avib_bC)
            Af_O = (Eform_O + O_Avib_T - 1/2*(-kB2*T*np.log(Ztrans_O2*Zvib_O2*Zrot_O2)))
            Af_OH = (Eform_OH + OH_Avib_T - 1/2*(-kB2*T*np.log(Ztrans_O2*Zvib_O2*Zrot_O2*Ztrans_H2*Zvib_H2*Zrot_H2)))
            Af_S = Ef_O_C

            covO.append(xn_O)
            covOH.append(xn_OH)
            Aform_O.append(Af_O)
            Aform_OH.append(Af_OH)
            Aform_S.append(Af_S)
    return [covO,covOH,Aform_O,Aform_OH,Aform_S]

In [6]:
npoints = 300
dpi_set = 300
T = [300,475,650,825,1000,1175,1350,1525,1700] # K, Temperature 
yH2 = 0.1
yO2 = [0.1]
P0 = 1*10**5 # Pa
phi = np.radians(np.linspace(0,360,npoints))
ph1,ph2 = np.meshgrid(phi,phi)
p = './'
surf_name = ''

# normalization constants based on facet surfae area relative to (111), order is in [(111), (100), (110)]
norm_cov = [1.0,0.865854575,0.61237259]

# mean-field parameters of Eads versus normalized coverage, order is in [(111), (100), (110)]
Osub_A = [2.142278,2.084212333,-0.047107333]
Osub_B = [-2.78671625,-2.930972333,-1.777542167]

Osurf_A = [3.449485,3.331004,1.074396333]
Osurf_B = [-2.56162575,-2.8476395,-1.592155583]

OHsub_B = [-2.34274225,-2.563163,-2.637266]
OHsurf_B = [-1.820411,-2.033224,-2.3253505]

# mean-field parameters of Eform versus normalized coverage, order is in [(111), (100), (110)]
Osub_C = [-8.9195,-10.5142,-11.6774]
Osub_D = [5.467,6.9551,9.2052]

Osurf_C = [-6.15,-8.0988,-9.5558]
Osurf_D = [4.3145,5.7443,8.5113]

OHsub_D = [3.12428145,4.3919727,6.5679803]
OHsurf_D = [2.4940727,3.7110857,6.1859668]

# multi-variable regression parameters for Gvib versus normalized coverage and T, order is in [(111), (100), (110)]
Osub_mA = [-7.711669E-07,-8.203429E-07,-8.339048E-07]
Osub_mB = [8.928464E-05,-7.901644E-05,2.226951E-07]
Osub_mC = [4.047748E-01,3.563822E-01,3.432920E-01]

Osub_bA = [-1.131690E-08,-1.053558E-08,1.331673E-09]
Osub_bB = [-3.255668E-05,-4.414758E-05,-1.169763E-04]
Osub_bC = [-1.122030E-02,-1.014872E-02,1.637662E-03]

Osurf_mA = [-8.096014E-07,-8.469408E-07,-8.157929E-07]
Osurf_mB = [-8.219717E-05,-2.242761E-04,-4.536756E-04]
Osurf_mC = [3.675412E-01,3.312777E-01,3.631082E-01]

Osurf_bA = [-2.484185E-09,5.825992E-09,-5.436181E-09]
Osurf_bB = [8.217412E-06,4.076405E-05,-1.789706E-05]
Osurf_bC = [-2.681782E-03,5.373688E-03,-5.431244E-03]

OHsub_bA = [-3.541014E-07,-3.359655E-07,-3.374997E-07]
OHsub_bB = [-9.180720E-05,-1.023675E-04,-1.124208E-04]
OHsub_bC = [3.661076E-01,3.827319E-01,3.825553E-01]

OHsurf_bA = [-3.525073E-07,-3.396716E-07,-3.404434E-07]
OHsurf_bB = [-1.274652E-04,-1.220683E-04,-1.449967E-04]
OHsurf_bC = [3.659180E-01,3.790999E-01,3.793502E-01]

# fit regression parameters to kubic harmonics for Eads and Gvib
# Eads
Osub_kub_A = kubic_harm(Osub_A,norm_cov)
Osub_kub_B = kubic_harm(Osub_B,[1,1,1])

Osurf_kub_A = kubic_harm(Osurf_A,norm_cov)
Osurf_kub_B = kubic_harm(Osurf_B,[1,1,1])

OHsub_kub_B = kubic_harm(OHsub_B,[1,1,1])
OHsurf_kub_B = kubic_harm(OHsurf_B,[1,1,1])

# Eform
Osub_kub_C = kubic_harm(Osub_C,norm_cov)
Osub_kub_D = kubic_harm(Osub_D,[1,1,1])

Osurf_kub_C = kubic_harm(Osurf_C,norm_cov)
Osurf_kub_D = kubic_harm(Osurf_D,[1,1,1])

OHsub_kub_D = kubic_harm(OHsub_D,[1,1,1])
OHsurf_kub_D = kubic_harm(OHsurf_D,[1,1,1])

# Gvib
Osub_kub_mA = kubic_harm(Osub_mA,norm_cov)
Osub_kub_mB = kubic_harm(Osub_mB,norm_cov)
Osub_kub_mC = kubic_harm(Osub_mC,norm_cov)

Osub_kub_bA = kubic_harm(Osub_bA,[1,1,1])
Osub_kub_bB = kubic_harm(Osub_bB,[1,1,1])
Osub_kub_bC = kubic_harm(Osub_bC,[1,1,1])

Osurf_kub_mA = kubic_harm(Osurf_mA,norm_cov)
Osurf_kub_mB = kubic_harm(Osurf_mB,norm_cov)
Osurf_kub_mC = kubic_harm(Osurf_mC,norm_cov)

Osurf_kub_bA = kubic_harm(Osurf_bA,[1,1,1])
Osurf_kub_bB = kubic_harm(Osurf_bB,[1,1,1])
Osurf_kub_bC = kubic_harm(Osurf_bC,[1,1,1])

OHsub_kub_bA = kubic_harm(OHsub_bA,[1,1,1])
OHsub_kub_bB = kubic_harm(OHsub_bB,[1,1,1])
OHsub_kub_bC = kubic_harm(OHsub_bC,[1,1,1])

OHsurf_kub_bA = kubic_harm(OHsurf_bA,[1,1,1])
OHsurf_kub_bB = kubic_harm(OHsurf_bB,[1,1,1])
OHsurf_kub_bC = kubic_harm(OHsurf_bC,[1,1,1])

# run monometallic NP model
file1 = open("CovSum_NiAu.txt", "w") # writes quantitative data to output file
graph_count = 1
for item in T:
    [covO_surf,covOH_surf,EO_surf,EOH_surf,ES_surf] = NP_calc(npoints,item,yO2[0],yH2,P0,Osurf_kub_A,Osurf_kub_C,
                                                              Osurf_kub_mA,Osurf_kub_mB,Osurf_kub_mC,Osurf_kub_bA,
                                                              Osurf_kub_bB,Osurf_kub_bC,Osurf_kub_C,Osurf_kub_D,
                                                              OHsurf_kub_C,OHsurf_kub_bA,OHsurf_kub_bB,OHsurf_kub_bC,
                                                              OHsurf_kub_D)
    [covO_sub,covOH_sub,EO_sub,EOH_sub,ES_sub] = NP_calc(npoints,item,yO2[0],yH2,P0,Osub_kub_A,Osub_kub_B,Osub_kub_mA,
                                                         Osub_kub_mB,Osub_kub_mC,Osub_kub_bA,Osub_kub_bB,Osub_kub_bC,
                                                         Osub_kub_C,Osub_kub_D,OHsub_kub_C,OHsub_kub_bA,OHsub_kub_bB,
                                                         OHsub_kub_bC,OHsub_kub_D)
    
    covO = []
    covOH = []
    layer = []
    for ii in range(0,npoints**2):
        covS_surf = 1 - covO_surf[ii] - covOH_surf[ii]
        covS_sub = 1 - covO_sub[ii] - covOH_sub[ii]
        Es_surf = (EO_surf[ii]*covO_surf[ii] + EOH_surf[ii]*covOH_surf[ii] + ES_surf[ii]*covS_surf)
        Es_sub = (EO_sub[ii]*covO_sub[ii] + EOH_sub[ii]*covOH_sub[ii] + ES_sub[ii]*covS_sub)
        if Es_sub < Es_surf:
            layer.append(0)
            covO.append(covO_sub[ii])
            covOH.append(covOH_sub[ii])
        else:
            layer.append(1)
            covO.append(covO_surf[ii])
            covOH.append(covOH_surf[ii])
    
    N = len(layer)
    aveO = sum(covO)/N
    aveOH = sum(covOH)/N
    ave_layer = sum(layer)/N*0.25
    file1.write(str(item)+" "+str(aveO)+" "+str(aveOH)+" "+str(ave_layer)+"\n")

    covO = np.reshape(covO,(npoints,npoints))
    covOH = np.reshape(covOH,(npoints,npoints))
    layer = np.reshape(layer,(npoints,npoints))

    NP_plot(ph1,ph2,covO,'CovO',p,surf_name,graph_count,0.0,0.98,dpi_set,'viridis')
    NP_plot(ph1,ph2,covOH,'CovOH',p,surf_name,graph_count,0.0,0.98,dpi_set,'viridis')
    NP_plot(ph1,ph2,layer,'Layer',p,surf_name,graph_count,0.0,0.98,dpi_set,'viridis')

    graph_count = graph_count + 1
    
    print("{:.1f}".format(item)+" completed!")
file1.close()

300.0 completed!
475.0 completed!
650.0 completed!
825.0 completed!
1000.0 completed!
1175.0 completed!
1350.0 completed!
1525.0 completed!
1700.0 completed!
